# Data wrangling con `{dplyr}` y `{tidyr}`

<div class="alert alert-block alert-info"> <b>NOTA:</b> Este tutorial asume un conocimiento básico de R. </div>

El término *data wrangling* no tiene una traducción directa al español, pero hace referencia a un proceso que consiste en transformar datos crudos en otro formato que pueda ser usado para su análisis, y por esta razón, tener un objetivo de análisis es importante, ya que con base en ese objetivo se planifica la manipulación de la estructura, granularidad, precisión, etc. Generalmente, implica alguno(s) de los siguientes pasos:

1. **Descubrir**: proceso de familiarización en el que se conceptualiza cómo podrían ser usados los datos, si existen variables incompletas, patrones reconocibles de errores, etc.
2. **Estructurar**: transformar para que pueda ser aprovechado más fácilmente (por ejemplo, usando el formato `tidy` o `long-format`, que veremos más adelante).
3. **Limpiar**: remover errores inherentes, que pueden surgir en varias formas. Por ejemplo, en bases de datos que han sido llenadas a mano por diferentes personas, una columna `Años` puede tener entradas que no sigan el mismo formato. Reducir los errores de tal forma que no influyan en los análisis.
4. **Enriquecer**: una vez que se entiende un conjunto de datos y se ha transformado a un conjunto usable, se puede tomar la decisión de si enriquecerlo con un conjunto de datos adicional.
5. **Validar**: verificar que el conjunto de datos es consistente y de calidad. Por ejemplo, si una columna en datos tabulares representa una variable, debe tener el mismo tipo de datos. Si una fila representa una observación, debe tener datos solo de esa observación, etc. Algunas reglas de consistencia son: tipo de datos (enteros, punto flotante, cadenas de caracteres), rango de datos (e.g., 0-100; por ejemplo, si una columna son Años, no puede tener valores negativos o mayores a 150), unicidad (e.g., códigos postales o un ID), consistencia de expresiones (e.g., usar solo una entre CP, C.P., Código Postal, CodPost, etc), valores nulos.

## `tidy` data

"A dataset is a collection of values, usually either numbers (if quantitative) or strings AKA text data (if qualitative). Values are organised in two ways. Every value belongs to a variable and an observation. A variable contains all values that measure the same underlying attribute (like height, temperature, duration) across units. An observation contains all values measured on the same unit (like a person, or a day, or a city) across attributes." Wickham, 2014

El formato [`tidy`](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html#:~:text=Tidy%20data%20is%20a%20standard,Every%20row%20is%20an%20observation.) es un estándar de presentación que mapea el significado de un conjunto de datos a su estructura. Consiste básicamente en las siguientes tres reglas:

1. Cada columna es una variable.
2. Cada fila es una observación.
3. Cada celda es un valor único.


<center><img src="/home/mrrobot/Documentos/GitHub/Analisis_multivariado/img/tidy-1.png" width=460/></center>

## `{dplyr}`

[`{dplyr}`](https://dplyr.tidyverse.org/) es un paquete para la manipulación de datos que consiste en un conjunto de 'verbos' (o funciones) como:

- `mutate()`: añade nuevas variables (columnas) 
- `select()`: seleccionar columnas (variables) de un `data frame` *por su nombre* y retorna un `data frame` con solo las columnas seleccionadas.
- `filter()`: selecciona *casos* (filas) por su valor. Necesita una expresión lógica con la cual realizar la selección.
- `summarise()`: crea un nuevo `data frame` basado en combinaciones únicas de variables agrupadoras.

## `{tidyr}`

Las funciones del paquete [`{tidyr}`](https://tidyr.tidyverse.org/) están en las siguientes categorías

- Pivotar: convertir datos entre formato largo y formato amplio. Se usan principalmente dos funciones
  - `pivot_longer()`, que incrementa el número de filas y reduce el número de columnas.
  - `pivot_wider()`, que hace lo contrario.
- Rectangular: que convierte listas de datos anidados (como en formato JSON) en tablas rectangulares. 
  - `unnest_longer()`: cambia cada elemento de una lista-columna en una fila.
  - `unnest_wider()`: cambia cada elemento de una lista-columna en una columna.
  

A continuación veremos cómo usar diversas funciones de `{dplyr}` y `{tidyr}` para dejar nuestros datos en el estandar de `tidy data`.

Comenzamos instalando el metapaquete `{tidyverse}`, que contiene a los paquetes `{dplyr}` y `{tidyr}` (además de otros que usaremos en otros tutoriales).

In [3]:
# correr solo una vez si no se tiene instalado ya
# install.packages('tidyverse') # descomentar para instalar, puede tardar

Para ejemplificar algunos usos, usaremos dos datasets que vienen junto con `{tidyverse}`:
- `mpg`: es un conjunto de datos de economía de combustible entre 1999 y 2008 para 38 modelos de automóviles. Contiene 11 variables:
  - manufacturer, model, displ (cilindrada), year, cyl (número de cilindros), trans (transmisión mecánica o automática), drv (tracción, 4 tipos), cty (millas (urbanas) recorridas por galón), hwy (millas (en carretera) recorridas por galón), fl (tipo de combustible), class (tipo de coche).
- `relig_income`: encuesta del Pew Research Center sobre la religión y el ingreso. Contiene las variables:
  - Religión (religión con la que se asocian) y 17 variables categóricas más sobre el ingreso (e.g., `<$10k`, `$10-20k`, etc.).

In [1]:
# Cargar el paquete tidyverse
suppressPackageStartupMessages(library(tidyverse, warn.conflicts = FALSE))
# visualizar las primeras 6 filas
head(mpg)

manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
<chr>,<chr>,<dbl>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>
audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
audi,a4,2.8,1999,6,manual(m5),f,18,26,p,compact


In [4]:
# crear una nueva variable de millas por galón que sea el promedio de millas
# en la ciudad y millas en carretera
mpg <- mpg %>%
   mutate(avg_mpg = (cty + hwy) / 2)
# visualizar las primeras 6 filas
head(mpg)

manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,avg_mpg
<chr>,<chr>,<dbl>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>
audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,23.5
audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,25.0
audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,25.5
audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,25.5
audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,21.0
audi,a4,2.8,1999,6,manual(m5),f,18,26,p,compact,22.0


[1] "audi"       "chevrolet"  "dodge"      "ford"       "honda"     
 [6] "hyundai"    "jeep"       "land rover" "lincoln"    "mercury"   
[11] "nissan"     "pontiac"    "subaru"     "toyota"     "volkswagen"

Para seleccionar una o más variables y que las retorne como `data.frame` o `tibble` usamos  `select()`.

In [11]:
mpg %>%
  select(manufacturer, cyl) %>%
  # para mostrar solo las primeras 6 filas
  head()

manufacturer,cyl
<chr>,<int>
audi,4
audi,4
audi,4
audi,4
audi,6
audi,6


In [10]:
# seleccinar manufacturer, cyl y displ y asignarlas a un objeto
mcd <- mpg %>%
  select(manufacturer, cyl, displ)
head(mcd)

manufacturer,cyl,displ
<chr>,<int>,<dbl>
audi,4,1.8
audi,4,1.8
audi,4,2.0
audi,4,2.0
audi,6,2.8
audi,6,2.8


In [12]:
# imprimir en consola y observar su estructura
head(relig_income)

religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k,$75-100k,$100-150k,>150k,Don't know/refused
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Agnostic,27,34,60,81,76,137,122,109,84,96
Atheist,12,27,37,52,35,70,73,59,74,76
Buddhist,27,21,30,34,33,58,62,39,53,54
Catholic,418,617,732,670,638,1116,949,792,633,1489
Don’t know/refused,15,14,15,11,10,35,21,17,18,116
Evangelical Prot,575,869,1064,982,881,1486,949,723,414,1529


Los nombres de las columnas no definen variables. De hecho, todos los nombres pertenecen a una variable (ingreso). Tenemos transformar el conjunto de datos de `wide` a `long` para eso, usamos `pivot_longer()` del paquete `tidyr`.

In [ ]:
relig_income %>% # función 'pipe'
  pivot_longer(
    cols = -religion, # todas excepto religion, más rápido
    names_to = 'income', # crea variable 'income' con nombres de columnas
    values_to = 'freq' # crea var 'freq' con los valores de las columnas
  ) 

# asignar a relig_income_tidy
relig_income_tidy <- relig_income %>% 
  pivot_longer(
    cols = -religion,
    names_to = 'income',
    values_to = 'freq'
  )

# ver primeros 6 de relig_income_tidy
head(relig_income_tidy)

## Valores perdidos explícitos e implícitos

Los valores perdidos *explícitos* son valores en donde aparece un `NA` para una combinación de filas y columnas. 

Los valores perdidos *implícitos* es la ausencia de un valor que debería estar. 

Una forma en la que los perdidos explícitos sucede de forma natural es cuando un valor de una celda se repite y, por lo tanto, se omite. Al cargarlo en R, por defecto se llena con `NA`.

In [27]:
treatment <- tribble(
  ~person,           ~treatment, ~response,
  "Derrick Whitmore", 1,         7,
  NA,                 2,         10,
  NA,                 3,         NA,
  "Katherine Burke",  1,         4
)
treatment

person,treatment,response
<chr>,<dbl>,<dbl>
Derrick Whitmore,1,7
NA,2,10
NA,3,NA
Katherine Burke,1,4


Si es el caso, esos valores se pueden rellenar con el último dato completo, una técnica que se llama, en inglés, *last observation carried forward* (LOCF).

In [29]:
treatment %>%
  fill(person)

person,treatment,response
<chr>,<dbl>,<dbl>
Derrick Whitmore,1,7
Derrick Whitmore,2,10
Derrick Whitmore,3,NA
Katherine Burke,1,4


Si queremos hacer lo mismo en todas las columnas (asumiendo que un dato repetido no se escriba en esas variables), usamos simplemente

In [30]:
treatment %>%
  fill(everything())

person,treatment,response
<chr>,<dbl>,<dbl>
Derrick Whitmore,1,7
Derrick Whitmore,2,10
Derrick Whitmore,3,10
Katherine Burke,1,4


In [31]:
stocks <- tibble(
  year  = c(2020, 2020, 2020, 2020, 2021, 2021, 2021),
  qtr   = c(   1,    2,    3,    4,    2,    3,    4),
  price = c(1.88, 0.59, 0.35,   NA, 0.92, 0.17, 2.66)
)
stocks

year,qtr,price
<dbl>,<dbl>,<dbl>
2020,1,1.88
2020,2,0.59
2020,3,0.35
2020,4,NA
2021,2,0.92
2021,3,0.17
2021,4,2.66


En este conjunto de datos, la variable `price` tiene un faltante explícito (`NA` en el cuarto trimestre del 2020), pero también tiene un faltante implícito en el primer trimestre del 2021. Usando `pivot_wider` (que cambia de `long` a `wider`) se puede hacer explícito, porque cada combinación de variables y unidades de observación (columnas y filas) debe tener un valor, por lo que la celda `[2021, 1]` se rellena con `NA`. 

In [26]:
stocks %>%
  pivot_wider(
    names_from = qtr,
    values_from = price
  )

year,1,2,3,4
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2020,1.88,0.59,0.35,NA
2021,NA,0.92,0.17,2.66


Por otro lado, si conocemos el valor que debería tener un faltante, podemos reemplazar el `NA` un valor explícito usando la función `complete()`. Los argumentos de esta función son `data` y las columnas que se quieren expandir por combinaciones únicas de valores. En este caso, queremos un valor de `price` por cada combinación única de `year` y `qtr` (trimestre). El tercer agrumento es `fill`, que espera una lista con nombres de las columnas a ser expandidas y los valores que debe sustituir en vez de `NA`, pero ojo: si el cuarto argumento, `explicit` se deja por defecto (`TRUE`) se reemplazan tanto los faltantes explícitos como implícitos. Cambiar `explicit` a `FALSE` provoca que *solo* los implícitos sean reemplazados por el valor indicado en `fill`.

In [24]:
stocks %>%
  complete(year, qtr, fill = list(price = 0), explicit = FALSE)

year,qtr,price
<dbl>,<dbl>,<dbl>
2020,1,1.88
2020,2,0.59
2020,3,0.35
2020,4,NA
2021,1,0.00
2021,2,0.92
2021,3,0.17
2021,4,2.66


In [25]:
# pivotar de nuevo
stocks %>%
  complete(year, qtr, fill = list(price = 0), explicit = FALSE) %>%
  pivot_wider(
    names_from = qtr,
    values_from = price
  )

year,1,2,3,4
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2020,1.88,0.59,0.35,NA
2021,0.00,0.92,0.17,2.66
